# Overview of the problem 

The risk of the borrower not repaying any type of loan is known as the **Credit Risk**. For *MoneyLion* they would be the *lender* and their customers would be the *borrowers*. Customers could range from Blue-collar workers to big corportate houses. An example of the problem would be me taking a *RM100,000* loan for *8 years* at *8% interest rate*. If I were to pay the installements for the first few years but then suddenly stopped the remaining unpaid installements would be considered a loss to *MoneyLion*. From a simple understanding this is bad news for *MoneyLion* as they lose money, but losses like this could increase exponentially as seen in the US 2008 recession where mortage home loans were given to individuals with a low credit score, while charging higher interest rates. Banks sold these loans to investors on the secondary market. As these home loans had been given to people with a higher credit risk, many were defaulting on them resulting in banks seizing their property. Resulting in a real estate bubble burst and a rapid decline in home prices. 

Therefore, for a bank ensuring that they assess the credit risk of their customers before approving loans as more defaults on loans would result in a decrease in interest rates on the deposits into banks. This hurts customers with good credit scores. 

## Business POV

MoneyLion would be dealing with two types of customers:

- Applicants that are likely to repay the loan
- Applicants that are likely to default on the loans

Outcomes of each scenario:

- MoneyLion makes profits as they are able to regain their loan + interest rate
- MoneyLion loses profits

Types of risks in each scenario:

- Bad credit score system that is too safe in making decisions might reject this customer resulting in a loss of revenue.
- Bad credit score system that is careless in making decisions might approve this customer resulting in a loss of revenue. 


## Business Objectives

Develop a loan repayment system that is able to properly strike a balance in ensuring that loans are given to customers who are going to pay back the loan and also ensuring loans to customers that are going to default on the loan are rejected. 

## Initial run through of the dataset


In [ ]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport

In [ ]:
# Imports and settings for pandas profiler
import sys

!{sys.executable} -m pip install -U pandas-profiling[notebook]
!jupyter nbextension enable --py widgetsnbextensio

In [ ]:
LOAN = "/content/drive/MyDrive/MoneyLionDataset/loan.csv"
PAYMENT = "/content/drive/MyDrive/MoneyLionDataset/payment.csv"
CLARITY = "/content/drive/MyDrive/MoneyLionDataset/clarity_underwriting_variables.csv"

In [ ]:
df_loan = pd.read_csv(LOAN, parse_dates=['applicationDate', 'originatedDate'])
df_payment = pd.read_csv(PAYMENT, parse_dates=['paymentDate'])
df_clarity = pd.read_csv(CLARITY)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9,11,12,13,14,15,16,17,18,19,20,21,22,23,25,26,27,28,29,31,32,33,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Details about the datasets:

- loan.csv has a **loanId** column to join with payment.csv where every row is a loan application or a successfully funded loan 
- payment.csv has a **loanId** column to join with loan.csv where every row represents an Automated Clearing House(ACH) transfers which are just electronic bank-to-bank money transfers
- clarity_underwriting_variables.csv has a column **underwritingid** that can be joined with loan.csv on **clarityFraudId**

Looking at payment.csv there are multiple values for each loanId which indicates that each loan can have multiple payments. When joining payment.csv with loan.csv I need to make sure that each loanId has all the installements and not just keep a single installement. 

1. Left join clarity.csv with loan.csv on underwritingid and clarityFraudId as underwritingid is possibly null in loan.csv
2. Left join the resulting csv file with payment.csv on loanId as payment.csv might have payments that are from different loans not in loan.csv. 

In [ ]:
def reduce_mem_usage(df,name):
  """
  Reduces the memory usage of a dataset 
  @args: 
  df: A pandas dataframe
  name: Name of the dataframe
  """
  numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
  start_mem = df.memory_usage().sum() / 1024**2    
  for col in df.columns:
      col_type = df[col].dtypes
      if col_type in numerics:
          c_min = df[col].min()
          c_max = df[col].max()
          if str(col_type)[:3] == 'int':
              if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                  df[col] = df[col].astype(np.int8)
              elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                  df[col] = df[col].astype(np.int16)
              elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                  df[col] = df[col].astype(np.int32)
              elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                  df[col] = df[col].astype(np.int64)  
          else:
              if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                  df[col] = df[col].astype(np.float16)
              elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                  df[col] = df[col].astype(np.float32)
              else:
                  df[col] = df[col].astype(np.float64)    
  end_mem = df.memory_usage().sum() / 1024**2
  print(f"{name}")
  print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
  return df

My notebook was crashing because I was using too much RAM so I reduced the memory usage of the dataframes before processing them. 


In [ ]:
df_loan = reduce_mem_usage(df_loan,"Loan")
df_payment = reduce_mem_usage(df_payment, "Payment")
df_clarity = reduce_mem_usage(df_clarity, "Clarity")

Loan
Mem. usage decreased to 575.76 Mb (23.4% reduction)
Payment
Mem. usage decreased to 26.30 Mb (38.5% reduction)
Clarity
Mem. usage decreased to 16.80 Mb (18.1% reduction)


In [ ]:
df_ = pd.merge(df_loan, df_clarity, left_on="clarityFraudId", right_on="underwritingid", how="left")
df = pd.merge(df_, df_payment, left_on="loanId", right_on="loanId", how="left")

The dataset has been joined and after checking through it each **loanId** that has multiple installements has multiple rows joined together with a total of 1.2 Million rows.

In [ ]:
JOINED_DF = "/content/drive/MyDrive/MoneyLionDataset/payment.csv"
df.to_csv(JOINED_DF)

## Quick Exploratory Data Analysis 

To get a better understanding of all the variables in the data and save some time I will be using [Pandas-Profiling](https://github.com/pandas-profiling/pandas-profiling) to generate a report of the dataset.

You can scroll through the widget or check in the repo for the file **report.html** to view it in the browser.

In [ ]:
df = pd.read_csv(JOINED_DF, index_col=[0])

In [ ]:
report = ProfileReport(df, title="Quick EDA", minimal=True, html={"style": {"full_width": True}})

In [ ]:
report.to_notebook_iframe()

Summarize dataset:   0%|          | 0/89 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# Save the report 
report.to_file("/content/drive/MyDrive/MoneyLionDataset/report.html")

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Exploring All Variables 

Now with the generated report I will be looking through some of the basic information about each column and see if I am able to find anything interesting to help with the main objective. 

> *Please do read this with the report.html on the side to refer to the visualisations generated*


1. **loadId** does have 47.1% distinct values as a single loan could have multiple payments tied to it. 
2. **payFrequency** - payments are made the most with biweekly plans then followed by weekly and monthly plans. These seem to be the preferred timelines for MoneyLion's users, maybe due to better interest rate options being given to biweekly and weekly payments but I cannot confirm it till I have data on interest rates offered by MoneyLion. 
3. **apr** is the Annual Percentage Rate which I thought would be in terms of percentages and not continuous values but according to this [link](https://www.valuepenguin.com/personal-loans/average-personal-loan-interest-rates) it seems that these are credit scores and the range of values in the chart seem to follow the range of values in a credit score. 
4. **originated** shows that about 56.6% of the loans get past the first stage of underwriting. I could naively assume that these 56.6% of loans are all approved but I am aware that they are more steps in the process that a loan can be denied. 
5. **nPaidOdd** shows that 88.5% of customers are first time applicants of a loan which would be the most difficult customers to predict whether or not they are able to pay back their loans compared to customers that have a history of not defaulting on their loans. 
6. **approved** shows that 56.2% of loans are actually approved compared to 56.6% that are approved at the first stage. 
7. **isFunded** shows that 54.8% of the loans are actually funded by MoneyLion in the end with the 1.4% difference with the **approved** status due to a customer voiding there loan after successfull approval. Need more information to understand what happens here.  
8. **loanStatus** shows that roughly 15.4% + 1.2%(Paid Off Loan and Settlement Paid Off) = 16.6% have paid off their loan which can be labelled as the positive class for this task. 36.8% of the loans have a *Withdrawn Application* status which would actually not be labelled as *Not Paid Off* as there wasn't even a loan approved. There are 21 disctinct values here and I will need more information on these values to truly differentiate between *Paid Off Loan* and *Not Paid Off Loan*. 
9. **loanAmount** shows the amount of money that the customers are asking for in their loan excluding the interest. The mean loan amount(USD) is 604, which even after adjusting for inflation is not a substantial amount. The top 3 amounts for a loan are 500, 400 and 300. I had a preconceived notion that loans were taken for much larger values mostly for mortages but these seem to be more for perhaps credit card debts, vehicle payments etc. More information on what the loans are for would help here. 
10. **state** shows the states in the USA which can be used for plotting the loans on a map. Maybe customers from certain states are more likely to default on their loans.
11. **leadType** shows that most of the customers come from a ping post which is bought and with 61.9% of the customers coming from it, its a good investment. Other areas such as *prescreen* showcase that maybe more work needs to be done on the mail campaigns to improve the number of leads generated. 
12. **leadCost** could do an analysis of which type of lead needs more work or perhaps to be dropped or invested more depending on the cost. 
13. **clearfraudscore** is not labelled in the assessment document. Looking through it, it seems to be a score of likelyhood of fraud, with a higher chance of fraud happening from the larger values. Need more information on this. 
14. **installmentIndex** shows the current installment number which varies from 1 to 105. 43.8% of these are missing values and these are numerical values so I am not sure how to impute the missing values as they would indicate that there is no loan given. 
15. **isCollection** shows that 1.1% of the customers face difficulty in making payments, these people would be more likely to default on a loan. 
16. **principal**, **fees** and **paymentAmount** are details regarding the payments made for an installment, customers would be making payments according to **payFrequency**. 
17. **paymentStatus** shows that 22.0% of the payments are *Cancelled* but I am not sure why it is labelled as such, is it due to the customer not being able to pay or due to MoneyLion cancelling it. Need to look at separating the statuses that indicate that the user defaulted on the loan and the statuses that show the user paid back the loan. 
18. **paymentReturnCode** seems to answer the questions from the previous column and will help in separating out the classes. 


I will need to properly distinguish between the *Paid Off Loan* class and *Defaulted On Loan* class. 
There are a lot of missing values that need to be filled in. 
Lots of the numerical values are skewed and need to be normalized perhaps with log normalization. I think that I will just need to use the *load.csv* data for modelling as loan repayment prediction is done when the user applies for the loan and not when they are making payments. Currently, the data is joined together which is beneficial for EDA. 
